Описание
В данном блокноте исследуется распределение уникальных идентификаторов клиентов (cn) и пользователей (user_id) по странам, основываясь на данных, полученных из client_ip. Также проводится анализ доли клиентов и пользователей, взаимодействующих с сервисом, основанным на наличии домена bablosoft в запросах (req_host).

Этапы
Подключение к ClickHouse: Настраивается безопасное соединение с базой данных zprx с использованием учетных данных из переменных окружения.

Формирование первого запроса:

Получение данных о cn и соответствующих странах с фильтрацией:
Период: август 2024 года.
Исключение запросов из России.
Исключение ошибочных запросов.
Выделение cn, связанного с сервисом bablosoft.
Формирование второго запроса:

Извлечение всех уникальных связей между user_id и cn из таблицы zprx_ds.device.
Объединение данных:

Объединение двух выборок по cn для анализа связей с учетом стран и пользователей.
Анализ данных:

Подсчет уникальных значений cn и user_id для каждой страны.
Вычисление доли идентификаторов (cn и user_id), связанных с сервисом bablosoft.
Формирование итоговой таблицы с результатами для топ-40 стран.

In [1]:
#Скрипт используется для поиска сразну нескольких сервисов: bablosoft, ipinfo, zennolab
#Для того, чтобы найти процент cn с сервисом по странам, достаточно подствить эти значеня в метод LIKE в запросе SQL

In [2]:
import os
from clickhouse_driver import Client
import pandas as pd

# Получаем пароль из переменной окружения
password = os.getenv("CLICKHOUSE_PASSWORD")

# Проверяем, что пароль не является None
if password is None:
    raise ValueError("Переменная окружения CLICKHOUSE_PASSWORD не установлена")

client = Client(
    host='ch.infra.zprx.net',
    port=2044,
    user='zprx_analytics',
    password=password,
    secure=True,
    verify=False,
    database='zprx'
)

try:
    query = """
SELECT cn,
       CASE 
           WHEN req_host LIKE '%bablosoft%' THEN cn 
           ELSE NULL 
       END AS cn_with_service,
       geoip2_country(client_ip) country
FROM 
     zprx.zproxy_reqs_v1__merge
WHERE toYYYYMM(ts_millis) = toYYYYMM(toDate('2024-08-01'))
  AND geoip2_country(client_ip) != 'Russia'
  AND error = '0' 
GROUP BY cn, 
         cn_with_service,
         country
    """

    # Execute the query and store the result in DataFrame
    result = client.execute(query)
    country_per_cn = pd.DataFrame(result, columns=["cn", "cn_with_service", "country"])

    # Print the results in table format
    print(country_per_cn)

finally:
    client.disconnect()

               cn cn_with_service         country
0      eyoIMBbFZD            None          Poland
1      nNKe5pN3AO            None  United Kingdom
2      JUCo9VlFzy            None         Finland
3      bkYvoHgxmr            None       Hong Kong
4      9uTHcpBvez            None           Chile
...           ...             ...             ...
30207  CoyDahffZl            None     Netherlands
30208  GgFeVgu47I            None                
30209  yZZ7Wx7ACX            None         Finland
30210  PBM0bMK61O            None         Germany
30211  kyRqTvda5C            None       Singapore

[30212 rows x 3 columns]


In [3]:
import os
from clickhouse_driver import Client
import pandas as pd

# Получаем пароль из переменной окружения
password = os.getenv("CLICKHOUSE_PASSWORD")

# Проверяем, что пароль не является None
if password is None:
    raise ValueError("Переменная окружения CLICKHOUSE_PASSWORD не установлена")

client = Client(
    host='ch.infra.zprx.net',
    port=2044,
    user='zprx_analytics',
    password=password,
    secure=True,
    verify=False,
    database='zprx'
)

try:
    query = """
SELECT user_id,
       cn
FROM zprx_ds.device
WHERE user_id is not NUll       
GROUP BY user_id,
         cn
    """

    # Execute the query and store the result in DataFrame
    result = client.execute(query)
    country_per_user_id = pd.DataFrame(result, columns=["user_id", "cn"])

    # Print the results in table format
    print(country_per_user_id)

finally:
    client.disconnect()

          user_id          cn
0      rbwnJagmNN  Cb9hjRB849
1      EI38Acy1Fu  mW3r5vCPDq
2      SDRGfJMG0v  MbYBbZMMZ4
3      pOu6lPJB7O  JyW7Ax7yo2
4      kEVUaSXFTd  qKI4DRKVch
...           ...         ...
70290  NOP9RAwaNo  vlJbrXnnce
70291  MjOvWPJctn  VFEm6CHvos
70292  WckroSFUsK  IAGNs9LuqA
70293  Yzz1odAD28  p8trySDwOz
70294  XoJdMcvjbY  jEjgMeckvY

[70295 rows x 2 columns]


In [4]:
user_id_cn_countries = pd.merge(country_per_cn, country_per_user_id, on='cn', how='left')


In [5]:
# Подсчет общего числа уникальных значений 'cn' для каждой страны
unique_cn_per_country = user_id_cn_countries.groupby('country')['cn'].nunique().reset_index(name='total_unique_cn')  # Группировка по стране и подсчет уникальных 'cn'

# Подсчет уникальных значений 'cn', где 'cn_with_service' не None, для каждой страны
unique_cn_with_service_per_country = user_id_cn_countries[user_id_cn_countries['cn_with_service'].notna()].groupby('country')['cn'].nunique().reset_index(name='unique_cn_with_service')  # Фильтрация по 'cn_with_service' и подсчет уникальных 'cn'

# Объединение данных и замена NaN на 0
country_cn_service_share = pd.merge(unique_cn_per_country, unique_cn_with_service_per_country, on='country', how='left').fillna(0)  # Объединение по стране и заполнение пропусков нулями

# Расчет доли значений с учетом сервиса
country_cn_service_share['share_cn_with_service (%)'] = (country_cn_service_share['unique_cn_with_service'] / country_cn_service_share['total_unique_cn']) * 100  # Доля в процентах

# Приведение к целочисленному типу для удобства
country_cn_service_share['unique_cn_with_service'] = country_cn_service_share['unique_cn_with_service'].astype(int)  # Приведение к целым числам
country_cn_service_share['share_cn_with_service (%)'] = country_cn_service_share['share_cn_with_service (%)'].astype(int)  # Приведение к целым числам

# Сортировка по доле в убывающем порядке и выбор топ-40
country_cn_service_share.sort_values(by='share_cn_with_service (%)', ascending=False).head(40)  # Сортировка и вывод топ-40


,country,total_unique_cn,unique_cn_with_service,share_cn_with_service (%)
131,Uruguay,81,72,88
25,Colombia,294,187,63
53,Indonesia,114,32,28
22,Canada,316,83,26
90,Netherlands,1107,258,23
89,Nepal,22,4,18
126,Turkey,380,65,17
62,Jordan,6,1,16
67,Kyrgyzstan,7,1,14
1,Afghanistan,19,2,10


In [6]:
# Подсчет общего числа уникальных значений 'user_id' для каждой страны
unique_user_id_per_country = user_id_cn_countries.groupby('country')['user_id'].nunique().reset_index(name='total_unique_user_id')  # Группировка по стране и подсчет уникальных user_id

# Подсчет уникальных значений 'user_id', где 'cn_with_service' не None, для каждой страны
unique_user_id_with_service_per_country = user_id_cn_countries[user_id_cn_countries['cn_with_service'].notna()].groupby('country')['user_id'].nunique().reset_index(name='unique_user_id_with_service')  # Фильтрация по cn_with_service и подсчет уникальных user_id

# Объединение данных в одну таблицу
country_user_service_share = pd.merge(unique_user_id_per_country, unique_user_id_with_service_per_country, on='country', how='left').fillna(0)  # Объединение по стране и заполнение пропусков нулями

# Расчет доли user_id с учетом сервиса в процентах
country_user_service_share['share_user_id_with_service (%)'] = (country_user_service_share['unique_user_id_with_service'] / country_user_service_share['total_unique_user_id']) * 100  # Доля в процентах

# Приведение типов данных для удобного отображения
country_user_service_share['unique_user_id_with_service'] = country_user_service_share['unique_user_id_with_service'].astype(int)  # Приведение к целым числам
country_user_service_share['share_user_id_with_service (%)'] = country_user_service_share['share_user_id_with_service (%)'].astype(int)  # Приведение к целым числам

# Сортировка данных по убыванию доли и вывод топ-40 записей
country_user_service_share.sort_values(by='share_user_id_with_service (%)', ascending=False).head(40)  # Сортировка и выбор топ-40



,country,total_unique_user_id,unique_user_id_with_service,share_user_id_with_service (%)
112,Slovenia,5,2,40
131,Uruguay,6,2,33
1,Afghanistan,4,1,25
62,Jordan,4,1,25
126,Turkey,83,20,24
90,Netherlands,147,27,18
89,Nepal,6,1,16
11,Bangladesh,19,3,15
42,Germany,242,36,14
67,Kyrgyzstan,7,1,14


In [7]:
country_cn_service_share.to_csv('доля_cn_с_bablosoft_по_странам(client_ip).csv', index=False)

In [8]:
country_user_service_share.to_csv('доля_user_id_с_bablosoft_по_странам(client_ip).csv', index=False)

Результаты
Для cn:

Вычислено общее количество уникальных cn и доля cn, связанных с сервисом.
Пример: Уругвай имеет 81 уникальный cn, из которых 72 связаны с bablosoft (88%).
Для user_id:

Вычислено общее количество уникальных user_id и доля тех, кто взаимодействовал с сервисом.
Пример: Словения имеет 5 уникальных user_id, из которых 2 связаны с bablosoft (40%).